In [7]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import pandas as pd
import numpy as np
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

import warnings

from src.features import build_features

warnings.filterwarnings("ignore")
sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)


In [9]:
df_procesada = pd.read_csv("../data/interim/datos_limpios.csv", encoding="latin-1", sep=";")
df_procesada["FECHA 1º evaluación"] = pd.to_datetime(
    df_procesada["FECHA 1º evaluación"], format="%Y-%m-%d"
)


In [19]:
egresos_por_anio = (
    df_procesada.groupby(["ANIO_PRIMERA_EVALUACION", "MES_PRIMERA_EVALUACION"])[
        "DIAGNOSTICO PRINCIPAL", "cie"
    ]
    .value_counts()
    .reset_index(name="conteo")
)

egresos_por_anio = build_features.agregar_info_codigo_cie(egresos_por_anio, "cie")
egresos_por_anio = egresos_por_anio.drop(columns=["Versión", "cie"])

In [20]:
egresos_por_anio.to_excel("../data/processed/conteo_diagnostico.xlsx", index=False)